In [1]:
import os
import copy
import numpy as np
import cv2
import h5py
from openvino.tools.pot import Metric,DataLoader
from openvino.tools.pot import IEEngine, load_model, save_model,compress_model_weights, create_pipeline

In [5]:
path = "FaceMask/filtering/"

In [3]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision
import mxnet as mx
import numbers
import torch

In [4]:
class MaskDatasetrec(Dataset):
    def __init__(self,rec_path,tv,aug=False,input_size=112):
        
        self.input_size = input_size
        
        if aug and tv=='train':
            self.transform = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize(self.input_size),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomRotation(10),
                        transforms.RandomAffine(degrees=0,translate=None,scale=None,shear=20),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

                    ])

        else:
            self.transform = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize(self.input_size),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ])


        path_rec = os.path.join(rec_path,"{}.rec".format(tv))
        path_idx = os.path.join(rec_path,"{}.idx".format(tv))
        self.imgrec = mx.recordio.MXIndexedRecordIO(path_idx, path_rec, 'r')

        s = self.imgrec.read_idx(0)
        header, _ = mx.recordio.unpack(s)
        self.imgidx = np.array(list(self.imgrec.keys))

        self.tv = tv


    def __getitem__(self,index):

        idx = self.imgidx[index]
        s = self.imgrec.read_idx(idx)
        header, img = mx.recordio.unpack(s)
        label = header.label
        
        image = mx.image.imdecode(img).asnumpy() # rgb
        target = int(label[0]) # label = [mask_lb, id_lb]

        image = self.transform(image)

        return image,None

    def __len__(self):
        return len(self.imgidx)


In [6]:
data_loader = MaskDatasetrec(path,'train_id_balanced',True,112)

In [7]:
q_params=[{
    "name":"DefaultQuantization",
    "params":{
        "target_device":"CPU",
        "preset":"performance",
        "stat_subset_size":300},
}]

In [8]:
from openvino.tools.pot import IEEngine, load_model, save_model,compress_model_weights, create_pipeline

In [9]:
model_config={
    "model":"classification/mbf2.xml",
    "weights":"classification/mbf2.bin"
}

In [10]:
engine_config={'device':'CPU',}

In [11]:
# 1. model load
model = load_model(model_config=model_config)

In [12]:
# 2. engine initialize
engine = IEEngine(config=engine_config, data_loader=data_loader)

In [13]:
# 3. pipeline
pipeline = create_pipeline(q_params,engine,)
compressed_model = pipeline.run(model=model)

In [14]:
# 4. (optional) .bin file size 를 줄이기 위해 model weight 를 compress
compress_model_weights(compressed_model)

In [17]:
# 5. save path/ model name 으로 ptq 한 model Save
compressed_model_paths = save_model(
    model=compressed_model,
    save_path = "./",
    model_name = "mbf_quantINT8",
)